In [57]:
import pandas as pd
import urllib.request, json
from datetime import datetime as dt
import time
from datetime import timedelta, date
import re
import pytz
import folium


#class to house common function inherited across weather classes
class Parent_class():
    def return_json(self, url_api):
        with urllib.request.urlopen(url_api) as url:
            data = json.load(url)
            return data

    #coordinates in the format of [[lat, lon]...]
    def _polygon_to_map_general(self, coordinates, zoom_start = 10, invert = False):
        if invert:
            coordinates = [item[::-1] for item in coordinates]
            
        lat_mean = sum([item[0] for item in coordinates])/len(coordinates)
        long_mean = sum([item[1] for item in coordinates ])/len(coordinates)
        m = folium.Map(location=[lat_mean, long_mean], zoom_start=zoom_start)
        folium.Polygon(
            locations = coordinates,  # Flip lon/lat to lat/lon
            color='blue',
            fill=True,
            fill_color='cyan',
            fill_opacity=0.5
        ).add_to(m)
        return(m)
    #parse times from entry of the form : 2024-12-10T16:00:00+00:00/PT1H
    def parse_valid_times(self,validtimes_str):
        t0 = dt.fromisoformat( validtimes_str.split('/')[0])
        #extract how many days and hour it is valid for
        times = re.findall( r'\d+', validtimes_str.split('/')[1])
        ################################################################
        ####                    Needs to be fixed                   ####
        ################################################################
        if len(times) == 1:
            if 'd' in validtimes_str:
                d = times[0]
                h = 0
            else:
                d = 0
                h = times[0]
        else:
            [d, h] = times
        t1 = t0 + timedelta(days = int(d), hours = int(h))
        return((t0, t1))
    
#data from 'forecast' argument of the initial JSON file
#url of the form https://api.weather.gov/gridpoints/PQR/{X},{Y}/forecast
class forecast(Parent_class):
    def __init__(self, forecast_url):
        self.forecast_url = forecast_url
        forecast_detailed = self.return_json(forecast_url)  
        self.forecast_detailed = forecast_detailed
        #reverse lat and long to be able to use with external functions
        self.coordinates = [item[::-1] for item in forecast_detailed['geometry']['coordinates'][0]]
        
        #forecast is provided as a list of dictionaries which are converted to columns of a dataframe here 
        df = pd.DataFrame(self.forecast_detailed['properties']['periods'])        
        df.drop(['number', 'endTime', 'temperatureTrend'], axis = 1, inplace = True)
        df['startTime'] = pd.to_datetime(df['startTime'])
        #convert date time to AM PM format for easier consumption
        df['startTime'] = df['startTime'].apply(lambda x : x.strftime('%Y-%m-%d %I:%M:%S %p'))
        #extract wind speed mins and max
        df['windSpeed_min'] = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[0]))
        df['windSpeed_max']  = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[-1]))
        df['probabilityOfPrecipitation'] = df['probabilityOfPrecipitation'].apply( lambda x: x['value'] / 100 if x['value'] is not None else 0 )
        #df.drop(['number', 'endTime', 'temperatureTrend', 'windSpeed'], axis = 1, inplace = True)
        df = df.set_index('startTime')

        self.fullForecast_df = df

        self.TempForecast   =   df[['temperature', 'temperatureUnit', 'isDaytime']]
        self.prob_precp     =   df[['probabilityOfPrecipitation', 'isDaytime']]        
        self.wind           =   df[['windSpeed_min', 'windSpeed_max', 'windDirection', 'isDaytime']]

#data from 'forecastHourly' argument of the initial JSON file
#url of the form https://api.weather.gov/gridpoints/{{STATION_CODE}}/{X},{Y}/forecast/hourly
class forecastHourly(Parent_class):
    def __init__(self, forecastHourly_url):
        self.forecastHourly_url = forecastHourly_url
        forecastHourly_detailed = self.return_json(forecastHourly_url)
        self.forecastHourly_detailed = forecastHourly_detailed
        #reverse lat and long to be able to use with external functions
        self.coordinates = [item[::-1] for item in forecastHourly_detailed['geometry']['coordinates'][0]]
        
        #forecast is provided as a list of dictionaries which are converted to columns of a dataframe here 
        df = pd.DataFrame(forecastHourly_detailed['properties']['periods'])
        df.drop(['number', 'name', 'endTime', 'temperatureTrend', 'detailedForecast'], axis = 1, inplace = True)
        df['startTime'] = pd.to_datetime(df['startTime'])
        df['startTime'] = df['startTime'].apply(lambda x : x.strftime('%Y-%m-%d %I:%M:%S %p'))
        df['probabilityOfPrecipitation'] = df['probabilityOfPrecipitation'].apply( lambda x: x['value'] / 100 if x['value'] is not None else 0 )
        df = df.set_index('startTime')

        self.fullForecast_hourly_df = df
        self.TempForecast_hourly   =   df[['temperature', 'temperatureUnit', 'isDaytime']]
        self.prob_precp_hourly     =   df[['probabilityOfPrecipitation']]

class forecastGridData(Parent_class):
    def __init__(self, forecastGridData_url):
        self.forecastGridData_url = forecastGridData_url
        forecastGridData_detailed = self.return_json(forecastGridData_url)
        self.forecastGridData_detailed = forecastGridData_detailed

        properties = forecastGridData_detailed['properties']

        df_temporary = pd.DataFrame()
        df_full = pd.DataFrame()
        vars = []
        categories = {'temperature': [],
              'humidity' : [],
              'wind' : [],
              'precipitation' : [],
              'snow' : []}
        
        for k in properties.keys():
            #ignore entries that are not variables or have no values
            cond = (type(properties[k]) != dict) or ('values' not in properties[k]) or len(properties[k]['values']) == 0
            if cond: continue
            variable = properties[k]

            df_temporary = pd.DataFrame(variable['values']).rename(columns={'value': k +"_value"})
            if 'uom' in variable:
                df_temporary[k+"_units"] = variable['uom'].split(':')[-1]

            df_temporary['validTime'] = pd.to_datetime(df_temporary['validTime'].apply(lambda x: x.split('/')[0]))
            #df_temporary['validTime'] = df_temporary['validTime'].apply(lambda x: str(x))
            df_temporary = df_temporary.set_index('validTime')
            df_full = pd.concat((df_full, df_temporary), axis = 1)

            for c in categories:
                if c in k.lower():
                    categories[c].append(k)
            vars.append(k)
        self.variables = vars
        self.categories = categories
        self.full_forecastGrid_df = df_full

class weather_details(Parent_class):
    def __init__(self, lat, long):
        start = time.time()
        self.lat = lat
        self.long = long
        self.url = "https://api.weather.gov/points/" + str(lat) + ',' + str(long)
        self.data = self.return_json(self.url)

        #Create forecast class using forecast URL
        #URL of the format - https://api.weather.gov/gridpoints/{{STATION_CODE}}/{{X}},{{Y}}/forecast
        self.forecast =forecast(self.data['properties']['forecast'])

        #url of the form https://api.weather.gov/gridpoints/{{STATION_CODE}}/{X},{Y}/forecast/hourly
        self.forecast_hourly = forecastHourly(self.data['properties']['forecastHourly'])

        #url of the form https://api.weather.gov/gridpoints/{{STATION_CODE}}/{X},{Y}
        self.forecastGridData = forecastGridData(self.data['properties']['forecastGridData'])

        self._properties()

    def polygon_to_map(self, zoom_start = 10):
        return(self._polygon_to_map_general(self.forecast.coordinates, zoom_start))

    def _properties(self):
        self.forecastOffice = self.data['properties']['cwa']
        #assuming that elevations is always in meters
        self.elevation = self.forecast.forecast_detailed['properties']['elevation']['value']
        self.elevation_unit = self.forecast.forecast_detailed['properties']['elevation']['unitCode'].split(':')[1]

        self.coordinates = self.forecast.coordinates

        #defining properties such as forecast generator type, forecast generating date etc...
        self.forecast_specifications = {'forecast' : {},
                                        'forecast_hourly' : {},
                                        'forecastGridData' : {},
                                        "GridDetails" : {}}
        #######################forecast############################
        properties = self.forecast.forecast_detailed['properties']
        self.forecast_specifications['forecast']['forecastGenerator'] = properties['forecastGenerator']
        self.forecast_specifications['forecast']['generatedAt'] = dt.fromisoformat(properties['generatedAt'])
        self.forecast_specifications['forecast']['updateTime'] = dt.fromisoformat(properties['updateTime'])

        validtimes_str = properties['validTimes']
        (t0, t1) = self.parse_valid_times(validtimes_str)
        self.forecast_specifications['forecast']['validTime_start'] = t0
        self.forecast_specifications['forecast']['validTime_ends']  = t1

        #####################forecastHourly########################
        properties = self.forecast_hourly.forecastHourly_detailed['properties']
        self.forecast_specifications['forecast_hourly']['forecastGenerator'] = properties['forecastGenerator']
        self.forecast_specifications['forecast_hourly']['generatedAt'] = dt.fromisoformat(properties['generatedAt'])
        self.forecast_specifications['forecast_hourly']['updateTime'] = dt.fromisoformat(properties['updateTime'])

        validtimes_str = properties['validTimes']
        (t0, t1) = self.parse_valid_times(validtimes_str)
        self.forecast_specifications['forecast_hourly']['validTime_start'] = t0
        self.forecast_specifications['forecast_hourly']['validTime_ends']  = t1

        #####################forecastGridData########################   
        properties = self.forecastGridData.forecastGridData_detailed['properties']
        keys = ['@type', 'forecastOffice', 'gridId', 'gridX', 'gridY']
        for k in keys:
            self.forecast_specifications['GridDetails'][k] = properties[k]
        self.forecast_specifications['forecastGridData']['updateTime'] = dt.fromisoformat(properties['updateTime'])

        validtimes_str = properties['validTimes']
        (t0, t1) = self.parse_valid_times(validtimes_str)
        self.forecast_specifications['forecastGridData']['validTime_start'] = t0
        self.forecast_specifications['forecastGridData']['validTime_ends']  = t1



In [59]:
def return_json(url_api):
    with urllib.request.urlopen(url_api) as url:
        data = json.load(url)
        return data
def weather_api_coordinates(lat, long):
    url_api = "https://api.weather.gov/points/" + str(lat) + ',' + str(long)
    return(return_json(url_api))

def polygon_to_map(coordinates): #coordinates in the format of [[lat, lon]...]
    m = folium.Map(location=coordinates[0], zoom_start=13)
    folium.Polygon(
        locations = coordinates,  # Flip lon/lat to lat/lon
        color='blue',
        fill=True,
        fill_color='cyan',
        fill_opacity=0.5
    ).add_to(m)
    return(m)



lat = 47.675908
long = -122.208008

#lat = 46.189070
#long = -122.184252

#data = weather_api_coordinates(lat, long)

In [60]:
test = weather_details(lat, long)
#test.polygon_to_map()
test.forecastGridData.full_forecastGrid_df
#test.forecast_specifications['forecastGridData']

,temperature_value,temperature_units,dewpoint_value,dewpoint_units,maxTemperature_value,maxTemperature_units,minTemperature_value,minTemperature_units,relativeHumidity_value,relativeHumidity_units,...,lightningActivityLevel_value,twentyFootWindSpeed_value,twentyFootWindSpeed_units,twentyFootWindDirection_value,twentyFootWindDirection_units,waveHeight_value,waveHeight_units,wavePeriod2_value,wavePeriod2_units,probabilityOfThunder_value
validTime,,,,,,,,,,,,,,,,,,,,,
2024-12-13 10:00:00+00:00,6.111111,degC,5.000000,degC,NaN,NaN,5.555556,degC,91.0,percent,...,1.0,5.556,km_h-1,160.0,degree_(angle),0.0,m,0.0,s,0.0
2024-12-13 11:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-13 12:00:00+00:00,5.555556,degC,4.444444,degC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.408,km_h-1,120.0,degree_(angle),NaN,NaN,NaN,NaN,NaN
2024-12-13 13:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-13 14:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-20 21:00:00+00:00,10.555556,degC,NaN,NaN,NaN,NaN,NaN,NaN,75.0,percent,...,NaN,NaN,NaN,100.0,degree_(angle),NaN,NaN,NaN,NaN,NaN
2024-12-20 22:00:00+00:00,NaN,NaN,6.666667,degC,NaN,NaN,NaN,NaN,74.0,percent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-20 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,percent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
test.polygon_to_map()